In [1]:
from pprint import pprint

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.ingestion.reader.byblock import construct_readers_byblock
from op_analytics.datapipeline.etl.ingestion.reader.request import BlockBatchRequest
from op_analytics.datapipeline.models.compute.modelspec import ModelsDataSpec
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

In [ ]:
model_name = "native_transfers"


# Prepare data raeders
data_spec = ModelsDataSpec(models=[model_name],
    root_path_prefix="blockbatch")
blockbatch_request = BlockBatchRequest.build(
    chains=["base"],
    range_spec="@20250622:+1",
    root_paths_to_read=data_spec.input_root_paths,
)
readers: list[DataReader] = construct_readers_byblock(
    blockbatch_request=blockbatch_request,
    read_from=DataLocation.GCS,
)


# Show details for the batch we are processing.
pprint(readers[0])

# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, auxiliary_templates = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0],  # use the first reader
)

In [3]:
# # Debug: Test template execution step by step
# print("=== DEBUG TEMPLATE EXECUTION ===")

# try:
#     # Try to execute the template
#     native_transfers = auxiliary_templates["native_transfers"].to_relation(
#         duckdb_context=ctx,
#         template_parameters={
#             "raw_traces": traces_view,
#         },
#     )
#     print("✅ Template executed successfully!")
#     print(f"Native transfers relation: {native_transfers}")
    
#     # Check if table was created
#     tables = ctx.client.sql("SHOW TABLES").df()
#     print(f"Tables after template execution:")
#     print(tables)
    
# except Exception as e:
#     print(f"❌ Template execution failed: {e}")
#     import traceback
#     traceback.print_exc()

In [4]:
# # Debug: Check what we have
# print("=== DEBUG SETUP ===")
# print(f"Input datasets: {list(input_datasets.keys())}")
# print(f"Auxiliary templates: {list(auxiliary_templates.keys())}")
# print(f"Context client: {ctx.client}")

In [5]:
# # Debug: Check if traces data exists
# print("=== DEBUG TRACES DATA ===")
# traces_view = input_datasets["ingestion/traces_v1"].create_view()
# print(f"Traces view created: {traces_view}")

# # Check if traces have data
# traces_count = ctx.client.sql(f"SELECT COUNT(*) as count FROM {traces_view}").pl().to_dicts()[0]["count"]
# print(f"Traces count: {traces_count}")

# # Check if traces have value_lossless
# sample_traces = ctx.client.sql(f"SELECT value_lossless FROM {traces_view} WHERE value_lossless IS NOT NULL AND value_lossless != '0' LIMIT 5").df()
# print(f"Sample value_lossless values:")
# print(sample_traces)

In [6]:
# # Debug: Test template execution step by step
# print("=== DEBUG TEMPLATE EXECUTION ===")

# try:
#     # Try to execute the template
#     native_transfers = auxiliary_templates["native_transfers"].to_relation(
#         duckdb_context=ctx,
#         template_parameters={
#             "raw_traces": traces_view,
#         },
#     )
#     print("✅ Template executed successfully!")
#     print(f"Native transfers relation: {native_transfers}")
    
#     # Check if table was created
#     tables = ctx.client.sql("SHOW TABLES").df()
#     print(f"Tables after template execution:")
#     print(tables)
    
# except Exception as e:
#     print(f"❌ Template execution failed: {e}")
#     import traceback
#     traceback.print_exc()

In [ ]:
print(ctx.client.sql("SHOW TABLES").df())

In [ ]:
traces_view = input_datasets["ingestion/traces_v1"].create_view()

native_transfers = auxiliary_templates["native_transfers"].to_relation(
    duckdb_context=ctx,
    template_parameters={
        "raw_traces": traces_view,
    },
)

In [ ]:
native_transfers = (
    native_transfers
    .filter("transfer_type = 'native'")
    .project("*")
)
df = ctx.client.sql(f"SELECT * FROM native_transfers").df()
df.head()

In [ ]:
# This matches the test's overall count query
block_filter_sql = "(block_number IN (128145990, 128145989) OR block_number % 100 < 2)"

num_native_transfers = ctx.client.sql(
    f"SELECT COUNT(*) as num_native_transfers FROM native_transfers WHERE {block_filter_sql}"
).pl().to_dicts()[0]["num_native_transfers"]
print("num_native_transfers:", num_native_transfers)

In [ ]:
df = ctx.client.sql(f"SELECT MIN(block_number) as min_bn, MAX(block_number) as max_bn FROM native_transfers").df()
df.head()

In [ ]:
df = ctx.client.sql(f"SELECT * FROM native_transfers WHERE block_number = 31880531 and transaction_hash = lower('0x1b85d1be582a90c2ae9682e0e1adf72f29e3aed609bbc57714f5676493716162') and to_address = lower('0x9c3631dDE5c8316bE5B7554B0CcD2631C15a9A05')").df()
df.head()

In [12]:
# df = ctx.client.sql(f"SELECT * FROM native_transfers WHERE block_number = 31880531").df()
# df.head()
# df.to_csv("native_transfers_31880531.csv", index=False)